In [1]:
import pandas as pd
import time
import re
import datetime

In [18]:
from selenium import webdriver
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
from konlpy.tag import Hannanum
from tqdm.notebook import tqdm

In [40]:
# 수집
def search(name_list):
    baseUrl = 'https://search.naver.com/search.naver?&where=news&query='
    url = baseUrl + urllib.parse.quote_plus(name_list) 
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    subject = []
    date = []
    contents = []
    company = []
    
    page = 1
    
    while True:
        
        url = baseUrl + urllib.parse.quote_plus(name_list) + '&start=' + str(page)
        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')

        word = soup.find_all(class_='type01')[0]
        word = word.find_all('dl')

        title = soup.find_all(class_='_sp_each_title')

        com = soup.find_all(class_='_sp_each_source')

        date_re = re.compile(r'\d\d\d\d.\d\d.\d\d.')
        day = soup.find_all(class_='txt_inline')


        # 컨텐츠
        for words in word:
            contents.append(words.find_all('dd')[1].get_text())

        # 타이틀
        for tie in title:
            subject.append(tie.get_text())

        # 업체
        for coms in com:
            if coms.get_text()[-2:] == '선정':
                company.append(coms.get_text()[:-6])
            else:
                company.append(coms.get_text())

        # 시간
        for dates in day:
            try:
                match = date_re.search(dates.get_text())
                date.append(match.group())
            except:
                date_re2 = re.compile(r'\d')
                match = date_re2.search(dates.get_text())
                m = int(match.group())
                year = str(datetime.datetime.today().year)
                month = str(datetime.datetime.today().month)
                day = str(datetime.datetime.today().day-m)
                result = year + '.' + month + '.' + day + '.'
                date.append(result)
        
        print(page, end=' ')
        
        try:
            if soup.find_all(class_='next')[0].get_text() != '다음페이지':
                break
            else: 
                page += 10
        except:
            if soup.find(class_='next') != '다음페이지':
                break

    df = pd.DataFrame(list(zip(subject, date, contents, company)), columns = ['title', 'date','contents', 'company'])
    
    return df

In [12]:
# 전처리 2
def sentence(df):
    
    df['contents'] = df['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]"," ")
    
    stopwords = ['을', '로', '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다''의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

    hannanum = Hannanum()
    
    contents = []

    for sentence in tqdm(df['contents']):
        temp_x = []
        temp_x = hannanum.nouns(sentence)
        temp_x = [word for word in temp_x if not word in stopwords]
        contents.append(temp_x)
        
    return contents


In [41]:
# 단어 빈도수
def word_count(contents, name):
    wordlist = sum(contents, [])
    word_list = pd.Series(wordlist)
    result = word_list.value_counts()
    result.to_csv('data/기업 '+name+'.csv', mode='w')
    print(name + ' Save compelte')

In [14]:
list_df = pd.read_csv('data/bankrup_list.csv')

In [15]:
# 회사명 리스트화
company_list = list_df['company'].tolist()

In [16]:
# 공백 없애기
company_list = [cmp_lst.strip() for cmp_lst in company_list]

In [44]:
result = pd.DataFrame(index=range(0,0), columns=['title', 'date','contents', 'company'])

In [43]:
# 리스트 기업 뉴스 한번에 스크롤링 해서 단어 빈도수 따로 저장
for i, cmp_lst in enumerate(company_list):
    print(cmp_lst + ' Start')
    df = search(cmp_lst)
    df_contents = sentence(df)
    word_count(df_contents, cmp_lst)
    result = pd.concat([result, df])

(유)동해건업 Start
1 11 21 31 41 51 61 71 81 91 101 111 121 131 141 151 161 171 181 191 201 211 221 231 241 251 261 271 281 291 301 311 321 331 341 351 361 371 381 391 401 411 421 431 441 451 461 471 481 491 


(유)동해건업 save compelte
(유)부강산업개발 Start
1 11 21 31 41 51 61 71 81 91 101 111 121 131 141 151 161 171 181 191 201 211 221 231 241 251 261 271 281 291 301 311 321 331 341 351 361 371 381 391 401 411 


(유)부강산업개발 save compelte
(유)세일디앤아이 Start
1 


(유)세일디앤아이 save compelte
(유)장수 Start
1 11 21 31 41 51 61 71 81 91 101 111 121 131 141 151 161 171 181 191 201 211 221 231 241 251 261 271 281 291 301 311 321 331 341 351 361 371 381 391 401 411 421 431 441 451 461 471 481 491 501 511 521 531 541 551 561 571 581 591 601 611 621 631 641 651 661 671 681 691 701 711 721 731 741 751 761 771 781 791 801 811 821 831 841 851 861 871 881 891 901 911 921 931 941 951 961 971 981 991 1001 1011 1021 1031 1041 1051 1061 1071 1081 1091 1101 1111 1121 1131 1141 1151 1161 1171 1181 1191 1201 1211 1221 1231 1241 1251 1261 1271 1281 1291 1301 1311 1321 1331 1341 1351 1361 1371 1381 1391 1401 1411 1421 1431 1441 1451 1461 1471 1481 1491 1501 1511 1521 1531 1541 1551 1561 1571 1581 1591 1601 1611 1621 1631 1641 1651 1661 1671 1681 1691 1701 1711 1721 1731 1741 1751 1761 1771 1781 1791 1801 1811 1821 1831 1841 1851 1861 1871 1881 1891 1901 1911 1921 1931 1941 1951 1961 1971 1981 1991 2001 2011 2021 2031 2041 2051 2061 2071 2081 2091 2101 2111 2121 2131 2141

KeyboardInterrupt: 

In [ ]:
result.to_csv('data/test.csv', mode='w', encoding='utf-8')